<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/BLOOM_Multilingual_Summarizer_%26_Reliable_Translations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate gradio newspaper3k lxml_html_clean langdetect -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 134.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 12.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [3]:
from newspaper import Article

In [4]:
from langdetect import detect, DetectorFactory

In [5]:
import gradio as gr,time,torch

In [6]:
DetectorFactory.seed = 0

In [7]:
BLOOM_MODEL = "bigscience/bloom-1b1"

In [8]:
MARIAN_MODELS = {
    "hi": "Helsinki-NLP/opus-mt-en-hi",
    "fr": "Helsinki-NLP/opus-mt-en-fr",
    "es": "Helsinki-NLP/opus-mt-en-es"
}

In [9]:
device = 0 if torch.cuda.is_available() else -1

In [10]:
tokenizer = AutoTokenizer.from_pretrained(BLOOM_MODEL)
model = AutoModelForCausalLM.from_pretrained(BLOOM_MODEL)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
marian_cache = {}

def load_marian(lang_code):
    """Load Marian model+tokenizer lazily and cache."""
    if lang_code in marian_cache:
        return marian_cache[lang_code]
    model_name = MARIAN_MODELS.get(lang_code)
    if not model_name:
        return None
    tok = AutoTokenizer.from_pretrained(model_name)
    mod = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    pipe = pipeline("translation", model=mod, tokenizer=tok, device=0 if torch.cuda.is_available() else -1)
    marian_cache[lang_code] = pipe
    return pipe

In [12]:
def fetch_article(url):
    try:
        art = Article(url)
        art.download()
        art.parse()
        txt = art.text
        if not txt or len(txt) < 50:
            return "Error: unable to extract enough article text. Try another URL or paste article text."
        return txt
    except Exception as e:
        return f"⚠️ Error fetching article: {e}"

In [13]:
def clean_generated_text(generated, prompt):
    # Remove the prompt prefix if present and strip whitespace
    if isinstance(generated, str):
        out = generated
    else:
        out = generated[0].get('generated_text', '')
    # sometimes the model echoes the prompt; remove the prompt if found
    if prompt and prompt in out:
        out = out.replace(prompt, '')
    return out.strip()

In [14]:
def summarize_text(text, max_tokens=140):
    prompt = f"Summarize the following news article in clear, concise English. Produce one short paragraph (3-5 sentences). Do NOT add your own opinions.\n\nArticle:\n{text}\n\nSummary:"
    # generate
    res = generator(prompt, max_new_tokens=max_tokens, temperature=0.3, top_p=0.9, do_sample=False)
    summary = clean_generated_text(res, prompt)
    # if model repeats the article or is too long, truncate heuristically to first 3 sentences
    sentences = summary.replace("\n", " ").split(". ")
    if len(sentences) > 4:
        summary = ". ".join(sentences[:3]).strip()
        if not summary.endswith("."):
            summary += "."
    return summary

In [15]:
def translate_with_bloom(english_text, target_lang_name, max_tokens=140):
    # Strict instruction to produce only translation in the target language, no extras
    prompt = (f"Translate the following English text into {target_lang_name}.\n"
              f"Return ONLY the translated text. Do NOT include the original English, explanations, or labels.\n\nText:\n{english_text}\n\nTranslation:")
    res = generator(prompt, max_new_tokens=max_tokens, temperature=0.2, top_p=0.9, do_sample=False)
    translated = clean_generated_text(res, prompt)
    return translated

In [16]:
def detect_language_safe(text):
    try:
        lang = detect(text)
        return lang
    except:
        return None

In [17]:
def translate_text_with_fallback(text, target_lang_name):
    # map language name to code
    name_to_code = {"Hindi":"hi","French":"fr","Spanish":"es","English":"en"}
    target_code = name_to_code.get(target_lang_name, None)
    if not target_code:
        # unknown target; attempt BLOOM translation anyway
        bloom_out = translate_with_bloom(text, target_lang_name)
        return bloom_out

    # First attempt: BLOOM strict prompt
    bloom_out = translate_with_bloom(text, target_lang_name)
    out_lang = detect_language_safe(bloom_out)
    # If bloom_out seems to be in the correct language (or detection uncertain), accept it
    if out_lang == target_code:
        return bloom_out

    # If BLOOM didn't produce desired language, fallback to Marian/OPUS models
    marian = load_marian(target_code)
    if marian:
        try:
            # Marian pipelines expect plain English input
            marian_res = marian(text)
            # pipeline returns a list of dicts or a string depending on pipeline version
            if isinstance(marian_res, list) and isinstance(marian_res[0], dict):
                return marian_res[0].get('translation_text', '').strip()
            elif isinstance(marian_res, str):
                return marian_res.strip()
            else:
                return str(marian_res).strip()
        except Exception as e:
            # if Marian fails, return BLOOM output but annotate
            return bloom_out + f"\n\n[Note: fallback translator failed: {e}]"
    else:
        # No Marian model for this language; return BLOOM output
        return bloom_out

In [18]:
def process_news(input_text_or_url):
    if input_text_or_url.strip().startswith("http"):
        text = fetch_article(input_text_or_url.strip())
    else:
        text = input_text_or_url.strip()

    if not text or len(text) < 80:
        return " Not enough content to summarize. Paste a longer article or valid URL.", "", "", ""

    # 1) Summarize
    summary = summarize_text(text)

    # 2) Translate summary into target languages with fallback if needed
    hindi = translate_text_with_fallback(summary, "Hindi")
    french = translate_text_with_fallback(summary, "French")
    spanish = translate_text_with_fallback(summary, "Spanish")

    return summary, hindi, french, spanish

In [19]:
app = gr.Interface(
    fn=process_news,
    inputs=gr.Textbox(label="Enter news article text or URL", lines=5, placeholder="Paste text or URL here..."),
    outputs=[
        gr.Textbox(label="English Summary"),
        gr.Textbox(label="Hindi Translation"),
        gr.Textbox(label="French Translation"),
        gr.Textbox(label="Spanish Translation")
    ],
    title="BLOOM Multilingual Summarizer & Reliable Translations",
    description="Summarizes articles with BLOOM and ensures correct translations (BLOOM primary, Marian models fallback)."
)


app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5cd83618c944c5472.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
